In [ ]:
# Clone this repo somewhere: https://github.com/mommermi/geotiff_sample

Find out where you are.

In [ ]:
!pwd

As talked about last week, you can also do the following:

In [ ]:
import os
where_i_am = os.getcwd()

Create a string that contains the path to the sample.tif

In [ ]:
import os
# Change these strings to fit where you know the file is!
where_the_geotiff_is = os.path.join(where_i_am, "repos", "geotiff_sample", "sample.tif")
print(where_the_geotiff_is)

Check if it is a file

In [ ]:
os.path.isfile(where_the_geotiff_is)

Now, let's have a look at gdal.

See here for more info: https://gdal.org/api/index.html#python-api

And here for an applied overview: http://slides.hannahaugustin.at/maptime/GDAL_intro/

In [ ]:
import sys
from osgeo import gdal

In [ ]:
# Check version of python:
print(sys.version)

# We can check which version we're running by printing the "__version__" variable
print("GDAL's version is: " + gdal.__version__)

# This shows where the version of GDAL is located (in this case, in a conda environment)
print(gdal)

# This allows GDAL to throw Python Exceptions
gdal.UseExceptions()

In [ ]:
info_options = gdal.InfoOptions(computeMinMax=True, stats=True)

print(gdal.Info(where_the_geotiff_is, options=info_options))

In [ ]:
# Open the file as a GDAL Dataset object.
in_ds = gdal.Open(where_the_geotiff_is, gdal.GA_ReadOnly)
print(in_ds)

# Close GDAL Dataset object.
in_ds = None

In [ ]:
in_ds = gdal.Open(where_the_geotiff_is, gdal.GA_ReadOnly)

# Find the number of bands in the dataset object.
print("[ NUMBER OF BANDS ] = ", in_ds.RasterCount)

# Create a GDAL raster band object from the only band in the dataset object.
in_band = in_ds.GetRasterBand(1)

print("[ BAND DATA TYPE ] = ", gdal.GetDataTypeName(in_band.DataType))
print("[ NO DATA VALUE ] = ", in_band.GetNoDataValue())

# Compute statistics if needed.
if in_band.GetMinimum() is None or in_band.GetMaximum()is None:
    in_band.ComputeStatistics(0)
    print("Statistics computed.")

print("[ OVERALL STATS ] = ", in_band.GetMetadata())

print("[ MIN ] = ", in_band.GetMinimum())
print("[ MAX ] = ", in_band.GetMaximum())
print("[ SCALE ] = ", in_band.GetScale())

# Close everything.
in_ds = None
in_band = None

Now, let's have a look at a specific image using rasterio. Rasterio is essentially what is called a wrapper for GDAL. This basically means that one if its main goals is to simplify using GDAL. See more on the philosophy here: https://rasterio.readthedocs.io/en/latest/intro.html

First, let's open the file using rasterio.open() (See https://rasterio.readthedocs.io/en/latest/api/rasterio.html#rasterio.open)

By default, GDAL drivers are tested sequentially until one works. You can simplify this by specifying a driver, if you like. Drivers can be found here: https://gdal.org/drivers/raster/index.html

In [ ]:
# Some of the next few cells are taken from this tutorial: https://medium.com/@mommermiscience/dealing-with-geospatial-raster-data-in-python-with-rasterio-775e5ba0c9f5
# and also from this one:
# First import the package.
import rasterio

In [ ]:
# We can check which version we're running by printing the "__version__" variable
print("rasterio's version is: " + rasterio.__version__)
print(rasterio)

First, try to open the file using the wrong driver to see what happens:

In [ ]:
tif_dataset = rasterio.open(where_the_geotiff_is, driver="BAG")

(Note, generally it is best practice in scripts to open files using "with" so that you don't have to worry about explicitly closing the files... For examples, see the rasterio.open() dics here: https://rasterio.readthedocs.io/en/latest/api/rasterio.html#rasterio.open)

In [ ]:
tif_dataset = rasterio.open(where_the_geotiff_is, driver="GTiff")

Let's investigate the python object we just created.

In [ ]:
print(tif_dataset)

Now, let's have a look at our opened object. Let's check the Coordinate Reference System:

In [ ]:
tif_dataset.crs

We have been given a rasterio.crs.CRS() object that includes an EPSG code. These can be explored at https://epsg.io, like here https://epsg.io/32631

In [ ]:
print(tif_dataset.crs)

Practice creating CRS objects using rasterio in a few different ways. See here for more info: https://rasterio.readthedocs.io/en/latest/api/rasterio.crs.html

In [ ]:
# The from_string method takes a variety of input.

crs = rasterio.crs.CRS.from_string("EPSG:3035")
print(crs)

In [ ]:
# EPSG codes may be used with the from_epsg method.
crs = rasterio.crs.CRS.from_epsg(3035)
print(crs)

In [ ]:
# The from_proj4 method takes PROJ strings as an argument
crs = rasterio.crs.CRS.from_proj4(proj="+proj=utm +zone=31 +datum=WGS84 +units=m +no_defs ")
print(crs)

Check the bounds of the dataset in the CRS coordinates.

In [ ]:
tif_dataset.bounds

In [ ]:
print(tif_dataset.bounds)

Checkout the affine transformation parameters. See https://github.com/rasterio/affine for more details...

In [ ]:
tif_dataset.transform

In [ ]:
# Read the dataset's valid data mask as a ndarray.
mask = tif_dataset.dataset_mask()
print(mask)

In [ ]:
import rasterio.features
import rasterio.warp
# Example from rasterio docs: https://rasterio.readthedocs.io/en/latest/index.html
# Extract feature shapes and values from the array.
for geom, val in rasterio.features.shapes(mask, transform=tif_dataset.transform):
    # Transform shapes from the dataset's own coordinate
    # reference system to CRS84 (EPSG:4326).
    geom = rasterio.warp.transform_geom(tif_dataset.crs, 'EPSG:4326', geom, precision=6)
    print(geom)

Ok, now, let's explore some of the base info we have about our dataset based on what we can access (see info on the base class here: https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html )

In [ ]:
tif_dataset.res

In [ ]:
tif_dataset.width

In [ ]:
tif_dataset.height

In [ ]:
tif_dataset.driver

In [ ]:
tif_dataset.interleaving

In [ ]:
tif_dataset.mode

In [ ]:
tif_dataset.count

In [ ]:
tif_dataset.descriptions

In [ ]:
tif_dataset.meta

Now let's look closer at the bands that are available. First, let's see what the file contains:

In [ ]:
tif_dataset.indexes

In [ ]:
img = tif_dataset.read(1)

In [ ]:
img

In [ ]:
type(img)

So, we have a numpy ndarray. What is that? See here: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html

If we look at the documentation, we can try to access a few things...

In [ ]:
img.shape

In [ ]:
img.dtype

In [ ]:
img.mean

In [ ]:
print(img.mean)

In [ ]:
img.mean()

In [ ]:
print(img.max())

In [ ]:
print(img.min())

In [ ]:
print(img.std())

Let's import numpy... Remember, you can also import using an alias.

In [ ]:
# No alias
import numpy
print(numpy.__version__)

# Alias or rename to "np" -- a very common practice
import numpy as np
print(np.__version__)


And let's also use rasterio's plotting functionality. See here for more info and examples: https://rasterio.readthedocs.io/en/latest/topics/plotting.html?highlight=plotting

In [ ]:
from rasterio.plot import show, adjust_band
# this bit of jupyter magic allows matplotlib to plot inline in a jupyter notebook
# See here: https://ipython.readthedocs.io/en/stable/interactive/plotting.html
%matplotlib inline  

imgdata = np.array([adjust_band(tif_dataset.read(i)) for i in (3,2,1)])
show(imgdata*3)  # factor 3 to increase brightness

In [ ]:
from rasterio import plot
import matplotlib.pyplot as plt
#multiple band representation
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))
plot.show(np.array(adjust_band(tif_dataset.read(3))), ax=ax1, cmap='Blues')
plot.show(np.array(adjust_band(tif_dataset.read(2))), ax=ax2, cmap='Greens')
plot.show(np.array(adjust_band(tif_dataset.read(1))), ax=ax3, cmap='Reds')
fig.tight_layout()

In [ ]:
#generate histogram
plot.show_hist(tif_dataset, bins=50, lw=0.0, stacked=False, alpha=0.3, histtype='stepfilled', title="Histogram")

... for those of you who are faster, you can continue working through this tutorial, which goes on to include transforming the CRS, cropping the image, resampling the image, and reprojecting the image: https://medium.com/@mommermiscience/dealing-with-geospatial-raster-data-in-python-with-rasterio-775e5ba0c9f5

In [ ]:
import sentinelhub
#
# For those who have an AWS account, you can try to navigate this and download a tile:
# From https://sentinelhub-py.readthedocs.io/en/latest/examples/aws_request.html#Data-into-.SAFE-structure
#